$$\LARGE
\underline
{
\textbf
    {Problema 2}
}
$$

Considere el entramado mostrado en la figura, con una fuerza aplicada de 20 kN. Calcule los
desplazamientos de cada uno de los nodos y las tensiones que sufre cada elemento. Todos los
elementos tienen $E = 210\ GPa$ y una sección de $10 cm^{2}$, excepto el elemento 3, que tiene una
sección de $20 cm^{2}$. Los elementos 2 y 5 tienen una longitud de 8 metros y el elemento 3 de 4
metros.

<center><img src="ImgProb02G02.png"></center>

In [1]:
import numpy as np
import MEF as mef

In [2]:
glxn = 2  # grados de libertad por nodo

In [3]:
MN = np.array([[0,0],[8,0],[16,0],[8,4]])  # pos de cada nodo  # metros
MC = np.array([[0,3],[0,1],[1,3],[2,3],[1,2]])  # ensamble entre elementos
E = np.array([210,210,210,210,210])*1e9  # Pascales
A = np.array([10,10,20,10,10])/(100**2)  # m2

s = np.array([0,1,5])
r = mef.vector_complemento(s, MN, glxn)

Fr = np.array([0,-20e3,0,0,0]).reshape([-1,1])
Us = np.array([0, 0, 0]).reshape([-1,1])

In [4]:
Nn = MN.shape[0]
Ne, Nnxe = MC.shape
glxn = 2

In [5]:
K, Ke = mef.Kglobal_barra(MN, MC, E, A, glxn)

In [6]:
F, U = mef.solve1D(K, r, s, Us, Fr)

In [7]:
np.set_printoptions(formatter={"float": lambda x: "%0.2f"% (x)})
F = F.reshape(Nn,glxn)
print('\n\033[4m'+f'Fuerzas x e y en cada nodo'+'\033[0m'+' [KN]:')
for n in range(Nn):
    print('\033[93m'+f'Nodo {n}: {F[n]*1e-3}'+'\033[0m')

U = U.reshape(Nn,glxn)
print('\n\033[4m'+f'Desplazamientos x e y de cada nodo'+'\033[0m'+' [mm]:')
for n in range(Nn):
    print('\033[93m'+f'Nodo {n}: {U[n]*1e3}'+'\033[0m')

K[np.abs(K) < 1e-16] = 0
print('\n\033[4m'+f'Matriz Global'+'\033[0m'+':\n\033[93m'+ f'K =\n {K/K.max()} \n'+'\033[0m')


Fuerzas x e y en cada nodo [KN]:
Nodo 0: [0.00 10.00]
Nodo 1: [0.00 -20.00]
Nodo 2: [0.00 10.00]
Nodo 3: [0.00 0.00]

Desplazamientos x e y de cada nodo [mm]:
Nodo 0: [0.00 0.00]
Nodo 1: [0.76 -3.84]
Nodo 2: [1.52 0.00]
Nodo 3: [0.76 -3.65]

Matriz Global:
K =
 [[0.39 0.08 -0.23 0.00 0.00 0.00 -0.16 -0.08]
 [0.08 0.04 0.00 0.00 0.00 0.00 -0.08 -0.04]
 [-0.23 0.00 0.46 0.00 -0.23 0.00 0.00 0.00]
 [0.00 0.00 0.00 0.92 0.00 0.00 0.00 -0.92]
 [0.00 0.00 -0.23 0.00 0.39 -0.08 -0.16 0.08]
 [0.00 0.00 0.00 0.00 -0.08 0.04 0.08 -0.04]
 [-0.16 -0.08 0.00 0.00 -0.16 0.08 0.33 -0.00]
 [-0.08 -0.04 0.00 -0.92 0.08 -0.04 -0.00 1.00]] 



Para calcular las tensiones en cada barra, hay que ver cuanto vario la longitud de cada una. 

Acomodo los desplazamientos de cada uno de los nodos en 2 dimensiones

In [8]:
U2D = U.reshape(Nn,glxn)

Calculo la posición final de los nodos:

In [9]:
MNf = MN + U2D

Calculo la longitud final $L_{f}$, y calculo las tensiones con la ecuación   $\sigma_{i}=E_{i}\ \varepsilon_{i}$ donde $i$ es cada uno de los elementos. 

In [10]:
sig = np.zeros([Ne,1])  # tensiones
for e in range(Ne):
    Lo = np.sqrt((MN[MC[e,1],0]-MN[MC[e,0],0])**2+(MN[MC[e,1],1]-MN[MC[e,0],1])**2)
    Lf = np.sqrt((MNf[MC[e,1],0]-MNf[MC[e,0],0])**2+(MNf[MC[e,1],1]-MNf[MC[e,0],1])**2)
    dL = Lf-Lo
    sig[e] = E[e]*dL

In [11]:
print('\033[4m'+f'Tensiones en cada barra'+'\033[0m'+' [MPa]:')
for e in range(Ne):
    print('\033[93m'+f'Elemento {e}: {sig[e]*1e-6}'+'\033[0m')

Tensiones en cada barra [MPa]:
Elemento 0: [-199.85]
Elemento 1: [160.19]
Elemento 2: [40.00]
Elemento 3: [-199.85]
Elemento 4: [160.19]
